In [1]:
#r"C:\Users\saifa\OneDrive\Desktop\AMD Dataset"

In [2]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Parameters
IMG_SIZE = (128, 128)  
NUM_CLASSES = 3  # Wet AMD, Dry AMD, Healthy

# Paths
train_path = r"C:\Users\saifa\OneDrive\Desktop\train"
val_path = r"C:\Users\saifa\OneDrive\Desktop\val"
results_of_gan=r"C:\Users\saifa\OneDrive\Desktop\Results_of_gan"

class_folders = {
    "Wet_AMD": "wetamd",
    "Dry_AMD": "dryamd",
    "Healthy": "healthy"
}

# Function to preprocess images
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return None
    img = cv2.resize(img, IMG_SIZE)
    img = img / 255.0  # Normalize
    return img

# Load dataset
def load_dataset(base_path):
    X, y = [], []
    for label, folder_name in enumerate(class_folders.values()):
        folder_path = os.path.join(base_path, folder_name)
        for file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, file)
            img = preprocess_image(img_path)
            if img is not None:
                X.append(img)
                y.append(label)
    return np.array(X), to_categorical(np.array(y), NUM_CLASSES)

# Load training and validation data
X_train, y_train = load_dataset(train_path)
X_val, y_val = load_dataset(val_path)

# Reshape for LSTM
X_train = X_train.reshape((X_train.shape[0], 1, IMG_SIZE[0], IMG_SIZE[1], 3))
X_val = X_val.reshape((X_val.shape[0], 1, IMG_SIZE[0], IMG_SIZE[1], 3))

print("Training data shape:", X_train.shape, "Labels:", y_train.shape)
print("Validation data shape:", X_val.shape, "Labels:", y_val.shape)


Training data shape: (360, 1, 128, 128, 3) Labels: (360, 3)
Validation data shape: (90, 1, 128, 128, 3) Labels: (90, 3)


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Augmentation
data_gen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Augment Training Data
augmented_images = []
augmented_labels = []

for i in range(len(X_train)):
    img = X_train[i].reshape(IMG_SIZE[0], IMG_SIZE[1], 3)  # Remove time-distributed dimension
    label = y_train[i]

    img = img.reshape((1,) + img.shape)  # Convert to (1, height, width, channels)

    for _ in range(3):  # Generate 3 augmented versions
        augmented_img = next(data_gen.flow(img, batch_size=1))[0]  # Keep the 4D format
        augmented_images.append(augmented_img)
        augmented_labels.append(label)

# Convert to numpy arrays
X_train = np.array(augmented_images + list(X_train.reshape(-1, IMG_SIZE[0], IMG_SIZE[1], 3)))
y_train = np.array(augmented_labels + list(y_train))

# Restore LSTM-compatible shape (5D)
X_train = X_train.reshape((X_train.shape[0], 1, IMG_SIZE[0], IMG_SIZE[1], 3))

print("New Training Data Size:", X_train.shape)


New Training Data Size: (1440, 1, 128, 128, 3)


In [4]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model

def improved_unet(input_size=(128, 128, 3)):
    inputs = Input(input_size)

    # Downsampling
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)

    # Upsampling
    u1 = UpSampling2D((2, 2))(c3)
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(c4)

    u2 = UpSampling2D((2, 2))(c4)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(c5)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c5)

    return Model(inputs, outputs)

# Compile & Train
model = improved_unet()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("Improved U-Net model loaded.")


Improved U-Net model loaded.


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Flatten, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D

# Improved Hybrid CNN-LSTM Model
model = Sequential([
    TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same'), input_shape=(1, IMG_SIZE[0], IMG_SIZE[1], 3)),
    TimeDistributed(BatchNormalization()),
    TimeDistributed(MaxPooling2D((2, 2))),
    
    TimeDistributed(Conv2D(64, (3, 3), activation='relu', padding='same')),
    TimeDistributed(BatchNormalization()),
    TimeDistributed(MaxPooling2D((2, 2))),

    TimeDistributed(Flatten()),

    LSTM(128, return_sequences=False),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile Model
import tensorflow as tf  # Ensure TensorFlow is imported

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', metrics=['accuracy'])

print("Improved CNN-LSTM Model Loaded.")


C:\Users\saifa\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Improved CNN-LSTM Model Loaded.


In [6]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

# Train
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_val, y_val),
                    callbacks=[early_stopping, reduce_lr])

print("Training Completed!")


Epoch 1/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 52s 498ms/step - accuracy: 0.5378 - loss: 0.9122 - val_accuracy: 0.3333 - val_loss: 1.2095 - learning_rate: 1.0000e-04
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 43s 473ms/step - accuracy: 0.7473 - loss: 0.6109 - val_accuracy: 0.3333 - val_loss: 1.3637 - learning_rate: 1.0000e-04
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 42s 469ms/step - accuracy: 0.7311 - loss: 0.5424 - val_accuracy: 0.3333 - val_loss: 1.4329 - learning_rate: 1.0000e-04
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 42s 467ms/step - accuracy: 0.7763 - loss: 0.4852 - val_accuracy: 0.3444 - val_loss: 1.3955 - learning_rate: 1.0000e-04
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 82s 467ms/step - accuracy: 0.8020 - loss: 0.4058 - val_accuracy: 0.5222 - val_loss: 0.8603 - learning_rate: 5.0000e-05
Epoch 6/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 42s 467ms/step - accuracy: 0.8234 - loss: 0.3957 - val_accuracy: 0.7556 - val_loss: 0.4280 - learning_rate: 5.0000e-05
Epoch 7/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 42s 466ms/step - acc

In [7]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.9198 - loss: 0.1779
Model Accuracy: 93.33%


In [8]:
def classify_image(image_path):
    img = preprocess_image(image_path)
    if img is None:
        return "Invalid image"
    img = img.reshape((1, 1, IMG_SIZE[0], IMG_SIZE[1], 3))
    prediction = model.predict(img)
    class_labels = ['Wet AMD', 'Dry AMD', 'Healthy']
    return class_labels[np.argmax(prediction)]

# Example usage
image_path = r"C:\Users\saifa\OneDrive\Desktop\AMD Dataset\Healthy Fundus\1ffa965b-8d87-11e8-9daf-6045cb817f5b..jpeg"
result = classify_image(image_path)
print("Predicted class:", result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 659ms/step
Predicted class: Healthy


In [9]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Generate predictions on validation/test data
y_pred = model.predict(X_val)  # Assuming X_val is your validation/test dataset
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels
y_true_classes = np.argmax(y_val, axis=1)  # True class labels

# Compute confusion matrix
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
print("Confusion Matrix:")
print(conf_matrix)

# Compute final test accuracy
accuracy = accuracy_score(y_true_classes, y_pred_classes)
print(f"\nFinal Test Accuracy: {accuracy:.4f}")

# Compute per-class metrics
precision = precision_score(y_true_classes, y_pred_classes, average=None)
recall = recall_score(y_true_classes, y_pred_classes, average=None)
f1 = f1_score(y_true_classes, y_pred_classes, average=None)

# Sensitivity (Recall) and Specificity Calculation
sensitivity = recall
specificity = []
for i in range(len(conf_matrix)):
    tn = np.sum(conf_matrix) - (np.sum(conf_matrix[i, :]) + np.sum(conf_matrix[:, i]) - conf_matrix[i, i])
    fp = np.sum(conf_matrix[:, i]) - conf_matrix[i, i]
    specificity.append(tn / (tn + fp))

# Dice coefficient calculation
dice_coeff = (2 * precision * recall) / (precision + recall + 1e-7)  # Avoid division by zero

# Print metrics for each class
class_names = ["Wet AMD", "Dry AMD", "Healthy"]
for i, class_name in enumerate(class_names):
    print(f"\nMetrics for {class_name}:")
    print(f"  Precision: {precision[i]:.4f}")
    print(f"  Sensitivity (Recall): {sensitivity[i]:.4f}")
    print(f"  Specificity: {specificity[i]:.4f}")
    print(f"  Dice Coefficient: {dice_coeff[i]:.4f}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 113ms/step
Confusion Matrix:
[[27  3  0]
 [ 3 27  0]
 [ 0  0 30]]

Final Test Accuracy: 0.9333

Metrics for Wet AMD:
  Precision: 0.9000
  Sensitivity (Recall): 0.9000
  Specificity: 0.9500
  Dice Coefficient: 0.9000

Metrics for Dry AMD:
  Precision: 0.9000
  Sensitivity (Recall): 0.9000
  Specificity: 0.9500
  Dice Coefficient: 0.9000

Metrics for Healthy:
  Precision: 1.0000
  Sensitivity (Recall): 1.0000
  Specificity: 1.0000
  Dice Coefficient: 1.0000


In [10]:
# Train U-Net model
EPOCHS = 20
BATCH_SIZE = 8

# Assuming X_train and Y_train contain segmented images and masks
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val, y_val))

# Save the trained U-Net model
model.save("unet_model.h5")
print("U-Net model saved as unet_model.h5")


Epoch 1/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 69s 361ms/step - accuracy: 0.9935 - loss: 0.0414 - val_accuracy: 0.9222 - val_loss: 0.1645
Epoch 2/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 65s 360ms/step - accuracy: 0.9925 - loss: 0.0433 - val_accuracy: 0.9222 - val_loss: 0.1867
Epoch 3/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 65s 360ms/step - accuracy: 0.9939 - loss: 0.0408 - val_accuracy: 0.9444 - val_loss: 0.1776
Epoch 4/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 65s 358ms/step - accuracy: 0.9951 - loss: 0.0327 - val_accuracy: 0.9222 - val_loss: 0.1815
Epoch 5/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 64s 358ms/step - accuracy: 0.9958 - loss: 0.0269 - val_accuracy: 0.9444 - val_loss: 0.1771
Epoch 6/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 64s 353ms/step - accuracy: 0.9966 - loss: 0.0253 - val_accuracy: 0.9222 - val_loss: 0.1681
Epoch 7/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 63s 347ms/step - accuracy: 0.9964 - loss: 0.0243 - val_accuracy: 0.9222 - val_loss: 0.1575
Epoch 8/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 63s 349ms/step - accuracy: 0.9981 - loss: 0

U-Net model saved as unet_model.h5


In [11]:

# Train Hybrid CNN-LSTM model
EPOCHS = 20
BATCH_SIZE = 8

model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val, y_val))

# Save the trained CNN-LSTM model
model.save("hybrid_model.h5")
print("Hybrid CNN-LSTM model saved as hybrid_model.h5")


Epoch 1/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 65s 360ms/step - accuracy: 0.9984 - loss: 0.0116 - val_accuracy: 0.9444 - val_loss: 0.1769
Epoch 2/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 64s 357ms/step - accuracy: 0.9987 - loss: 0.0070 - val_accuracy: 0.9333 - val_loss: 0.2209
Epoch 3/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 64s 358ms/step - accuracy: 0.9991 - loss: 0.0074 - val_accuracy: 0.9556 - val_loss: 0.1968
Epoch 4/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 64s 356ms/step - accuracy: 0.9999 - loss: 0.0061 - val_accuracy: 0.9222 - val_loss: 0.2423
Epoch 5/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 65s 359ms/step - accuracy: 0.9990 - loss: 0.0086 - val_accuracy: 0.9222 - val_loss: 0.2313
Epoch 6/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 65s 359ms/step - accuracy: 1.0000 - loss: 0.0059 - val_accuracy: 0.9222 - val_loss: 0.2067
Epoch 7/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 65s 359ms/step - accuracy: 0.9989 - loss: 0.0075 - val_accuracy: 0.9222 - val_loss: 0.2334
Epoch 8/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 82s 358ms/step - accuracy: 0.9979 - loss: 0

Hybrid CNN-LSTM model saved as hybrid_model.h5


In [18]:
# # After training
# model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
# model.save("model.h5")  # Ensures compilation info is saved

In [19]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.filters import sobel
from scipy.stats import entropy
from tqdm import tqdm

# === Metric Functions ===
def edge_strength(image):
    return np.mean(sobel(image))

def image_entropy(image):
    histogram, _ = np.histogram(image.flatten(), bins=256, range=(0, 1), density=True)
    return entropy(histogram + 1e-10)  # Avoid log(0)

def mean_intensity(image):
    return np.mean(image)

def foreground_ratio(image, threshold=0.5):
    return np.sum(image > threshold) / image.size

# === Process Folder Function ===
def evaluate_folder(folder_path, class_name):
    results = []
    for filename in tqdm(os.listdir(folder_path), desc=f"Processing {class_name}"):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif')):
            img_path = os.path.join(folder_path, filename)

            # Read and normalize image
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            img = img / 255.0  # Normalize to [0,1]

            # Compute metrics
            result = {
                "Class": class_name,
                "Image Name": filename,
                "Edge Strength": edge_strength(img),
                "Entropy": image_entropy(img),
                "Mean Intensity": mean_intensity(img),
                "Foreground Ratio": foreground_ratio(img)
            }

            results.append(result)

    return results

# === Main Evaluation ===
def evaluate_all_classes(wet_path, dry_path, healthy_path, output_csv='unet_evaluation_report.csv'):
    all_results = []

    all_results += evaluate_folder(wet_path, "WetAMD")
    all_results += evaluate_folder(dry_path, "DryAMD")
    all_results += evaluate_folder(healthy_path, "Healthy")

    df = pd.DataFrame(all_results)
    df.to_csv(output_csv, index=False)
    print(f"\n✅ Metrics saved to {output_csv}")

    # Print class-wise averages
    print("\n--- Average Metrics Per Class ---")
    class_avg = df.groupby("Class").mean(numeric_only=True)
    print(class_avg.round(4))

# === USAGE ===
# Replace these with your actual paths
wet_path     = r"C:\Users\saifa\OneDrive\Desktop\seg_imgs_colour\wetamd"
dry_path     = r"C:\Users\saifa\OneDrive\Desktop\seg_imgs_colour\dryamd"
healthy_path = r"C:\Users\saifa\OneDrive\Desktop\seg_imgs_colour\healthy"

evaluate_all_classes(wet_path, dry_path, healthy_path)


Processing Healthy: 100%|███████████████████████████████████████████████████████████| 120/120 [00:00<00:00, 250.89it/s]



✅ Metrics saved to unet_evaluation_report.csv

--- Average Metrics Per Class ---
         Edge Strength  Entropy  Mean Intensity  Foreground Ratio
Class                                                            
DryAMD          0.0273   4.5834          0.3215            0.1432
Healthy         0.0371   4.5436          0.3431            0.1973
WetAMD          0.0280   4.6013          0.3168            0.1447


In [21]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load the trained model
model = load_model("unet_model.h5")

# Define Labels (Modify as per your model)
labels = ["Wet AMD", "Healthy Fundus", "Dry AMD"]  # Replace with actual class names

# Function to Process Input and Predict
def predict(img):
    img = img.convert("RGB")  # Ensure image is in RGB format
    img = img.resize((128, 128))  # Resize to match model input size
    img_array = np.array(img, dtype=np.float32) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = np.expand_dims(img_array, axis=0)  # Add time-step dimension

    print("Processed image shape:", img_array.shape)  # Debugging

    # Make prediction
    predictions = model.predict(img_array)
    predicted_class = labels[np.argmax(predictions)]
    confidence = np.max(predictions) * 100
    return f"\n🧠 *Prediction:* {predicted_class}\n"# 🎯 *Confidence:* {confidence:.2f}% 
   

# Create Gradio Interface
iface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label="📤 Upload an Image"),
    outputs=gr.Textbox(label="📊 Prediction Results"),
    title="🔍 Hybrid Model Prediction",
    theme="soft",
)

iface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://eb123f75b5825309c5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Processed image shape: (1, 1, 128, 128, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
Processed image shape: (1, 1, 128, 128, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Processed image shape: (1, 1, 128, 128, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
Processed image shape: (1, 1, 128, 128, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


In [15]:
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load model
model = load_model("model.h5")

# Define class labels (modify if needed)
labels = ["Wet AMD", "Healthy Fundus", "Dry AMD"]

def predict_from_path(img_path):
    # Load and preprocess image
    img = Image.open(img_path).convert("RGB")  # Ensure RGB
    img = img.resize((128, 128))  # Resize to match model input
    img_array = np.array(img, dtype=np.float32) / 255.0  # Normalize
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = np.expand_dims(img_array, axis=0)  # Add time-step dimension (1, 128, 128, 3)

    print("Processed image shape:", img_array.shape)  # Debugging

    # Make prediction
    predictions = model.predict(img_array)
    predicted_class = labels[np.argmax(predictions)]
    confidence = np.max(predictions) * 100
    return f"🧠 *Prediction:* {predicted_class}\n🎯 *Confidence:* {confidence:.2f}%"

# Example usage
# image_path = r"C:\Users\saifa\OneDrive\Desktop\AMD Dataset\Healthy Fundus\1ffa962b-8d87-11e8-9daf-6045cb817f5b..jpeg"  # Replace with actual file path
# result = predict_from_path(image_path)
# print(result)


In [16]:
#######
import gradio as gr
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model

# Load the trained CNN-LSTM model
model = load_model("model.h5")  # Make sure this is the correct final hybrid model path

# Class labels (make sure they match model training order)
labels = ["Wet AMD", "Dry AMD", "Healthy"]

# Define image input size
IMG_SIZE = (128, 128)

def predict(img: Image.Image):
    # Preprocess the uploaded image
    img = img.convert("RGB")
    img = img.resize(IMG_SIZE)
    img_array = np.array(img, dtype=np.float32) / 255.0  # Normalize to [0, 1]

    # Reshape to match CNN-LSTM input shape: (batch, time_step, height, width, channels)
    img_array = img_array.reshape(1, 1, IMG_SIZE[0], IMG_SIZE[1], 3)

    # Predict
    predictions = model.predict(img_array)
    predicted_class = labels[np.argmax(predictions)]
    confidence = np.max(predictions) * 100

    # Return result
    return f"🧠 *Prediction:* {predicted_class}\n🎯 *Confidence:* {confidence:.2f}%"

# Gradio UI setup
iface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label="📤 Upload Fundus Image"),
    outputs=gr.Textbox(label="📊 Prediction"),
    title="🔍 AMD Classifier | CNN-LSTM Model",
    description="Upload a fundus image to classify it as Wet AMD, Dry AMD, or Healthy Retina.",
    theme="soft"
)

iface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://d87113bc5ad91bb240.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [17]:
import gradio as gr
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model

# —— Load your trained CNN-LSTM model —— 
model = load_model("model.h5")

# —— Make sure this matches your training order! —— 
labels = ["Wet AMD", "Dry AMD", "Healthy"]

IMG_SIZE = (128, 128)

def classify_fn(img: Image.Image):
    # Preprocess
    img = img.convert("RGB").resize(IMG_SIZE)
    arr = np.array(img, dtype=np.float32) / 255.0
    arr = arr.reshape(1, 1, IMG_SIZE[0], IMG_SIZE[1], 3)

    # Predict
    preds = model.predict(arr)
    idx = np.argmax(preds)
    conf = float(np.max(preds) * 100)

    return f"🧠 **Prediction:** {labels[idx]}\n🎯 **Confidence:** {conf:.2f}%"

# —— Build the Gradio Blocks interface —— 
with gr.Blocks(theme="soft") as demo:
    # Title + description
    gr.Markdown("## 🔎 AMD Classifier | CNN-LSTM Model")
    gr.Markdown("Upload a fundus image to classify it as Wet AMD, Dry AMD, or Healthy Retina.")

    with gr.Row():
        # Left column: upload + clear
        with gr.Column(scale=1):
            inp = gr.Image(type="pil", label="📤 Upload Fundus Image", elem_id="upload-box")
            clr = gr.Button("Clear", elem_id="clear-btn")
        # Right column: prediction + flag
        with gr.Column(scale=1):
            out = gr.Textbox(label="📊 Prediction", interactive=False, elem_id="prediction-box")
            flag = gr.Button("Flag", elem_id="flag-btn")

    # Submit below both columns
    submit = gr.Button("Submit", variant="primary", elem_id="submit-btn")

    # —— Wiring up events —— 
    submit.click(fn=classify_fn, inputs=inp, outputs=out)
    clr.click(lambda: None, None, inp)  # clears the image input

# Launch with shareable link
if __name__ == "__main__":
    demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://a7ab30e3555b978926.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
